In [ ]:
! curl http://localhost:8080/embeddings -X POST -H "Content-Type: application/json" -d '{"input": "Your text string goes here", "model": "text-embedding-ada-002"}'

In [5]:
! curl http://localhost:8080/v1/chat/completions -H "Content-Type: application/json" -d '{ "model": "mixtral", "messages": [{"role": "user", "content": "How are you?"}], "temperature": 0.9 }'

{"created":1708158369,"object":"chat.completion","id":"488992f7-5299-4bf5-b173-19a3057d49a6","model":"mixtral","choices":[{"index":0,"finish_reason":"stop","message":{"role":"assistant","content":"I'm just a computer program, so I don't have feelings or emotions. I'm here to help answer your questions to the best of my ability. Is there something specific you would like to know?\n\nIf you're asking about the status of my operation, I'm currently functioning as intended. Thank you for asking! Is there anything I can help you with?"}}],"usage":{"prompt_tokens":0,"completion_tokens":0,"total_tokens":0}}

# Langchain Function Calling

In [ ]:
!pip install langchain langchain-openai

In [7]:
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [8]:
os.environ["OPENAI_API_BASE"] = "http://localhost:8080/v1"
os.environ["OPENAI_API_KEY"] = "NONE"

In [9]:
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="mixtral")

In [10]:
output_parser = StrOutputParser()

In [11]:
chain = prompt | model | output_parser

In [12]:
out = chain.invoke({"topic": "ice cream"})

In [13]:
print(out)

Why don't ice creams ever argue?

Because they always cool down before things heat up!


In [14]:
(prompt | model).invoke({"topic": "ice cream"})

AIMessage(content="Why don't ice creams ever argue?\n\nBecause they always cool down before things heat up!")

In [15]:
(prompt).invoke({"topic": "ice cream"})

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [1]:
import openai
import json
import requests

from openai import OpenAI

In [2]:
client = OpenAI(base_url="http://localhost:8080/v1", api_key="NONE")


In [7]:
def get_current_weather(location, units="metric"):
    """Get the current weather in a given location"""

    url = f"https://api.tomorrow.io/v4/weather/realtime?location={location.lower()}&units={units}&apikey=hZdpIwxHljGjU8UCsACtOQla3Ct4mZs2"

    headers = {"accept": "application/json"}
    
    response = requests.get(url, headers=headers).json()
    
    weather_info = response['data']
    return json.dumps(weather_info)

In [11]:
get_current_weather("silchar")

'{"time": "2024-02-22T07:49:00Z", "values": {"cloudBase": 0.23, "cloudCeiling": 0.23, "cloudCover": 99, "dewPoint": 20.38, "freezingRainIntensity": 0, "humidity": 90, "precipitationProbability": 100, "pressureSurfaceLevel": 1005.54, "rainIntensity": 1.21, "sleetIntensity": 0, "snowIntensity": 0, "temperature": 24.88, "temperatureApparent": 24.88, "uvHealthConcern": 1, "uvIndex": 2, "visibility": 7.51, "weatherCode": 4200, "windDirection": 219.69, "windGust": 2.13, "windSpeed": 0.63}}'

In [12]:
functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. 'san francisco' or 'new york' or 'bengaluru' or 'toronro'",
                    },
                    "units": {"type": "string", "enum": ["metric", "imperial"]},
                },
                "required": ["location"],
            },
        }
    ]

In [14]:
messages = [{"role": "user", "content": "What's the weather like in Silchar?"}]

In [15]:
response = client.chat.completions.create(
    model="mixtral",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)

response_message = response.choices[0].message

In [17]:
response_message.function_call

FunctionCall(arguments='{"location":"Silchar","units":"metric"}', name='get_current_weather', function='get_current_weather')

In [18]:
if response_message.function_call:
    available_functions = {
        "get_current_weather": get_current_weather,
    }  # only one function in this example, but you can have multiple
    function_name = response_message.function_call.name
    fuction_to_call = available_functions[function_name]
    function_args = json.loads(response_message.function_call.arguments)
    function_response = fuction_to_call(
        **function_args,
    )

In [19]:
function_response

'{"time": "2024-02-22T08:09:00Z", "values": {"cloudBase": 0.21, "cloudCeiling": 0.21, "cloudCover": 100, "dewPoint": 20.19, "freezingRainIntensity": 0, "humidity": 91, "precipitationProbability": 100, "pressureSurfaceLevel": 1005.51, "rainIntensity": 2.45, "sleetIntensity": 0, "snowIntensity": 0, "temperature": 24.81, "temperatureApparent": 24.81, "uvHealthConcern": 1, "uvIndex": 2, "visibility": 4.17, "weatherCode": 4200, "windDirection": 302.31, "windGust": 2.31, "windSpeed": 1.63}}'

In [20]:
if response_message.function_call:
    messages.append(response_message)  # extend conversation with assistant's reply
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )  # extend conversation with function response
messages

[{'role': 'user', 'content': "What's the weather like in Silchar?"},
 ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Silchar","units":"metric"}', name='get_current_weather', function='get_current_weather'), tool_calls=None),
 {'role': 'function',
  'name': 'get_current_weather',
  'content': '{"time": "2024-02-22T08:09:00Z", "values": {"cloudBase": 0.21, "cloudCeiling": 0.21, "cloudCover": 100, "dewPoint": 20.19, "freezingRainIntensity": 0, "humidity": 91, "precipitationProbability": 100, "pressureSurfaceLevel": 1005.51, "rainIntensity": 2.45, "sleetIntensity": 0, "snowIntensity": 0, "temperature": 24.81, "temperatureApparent": 24.81, "uvHealthConcern": 1, "uvIndex": 2, "visibility": 4.17, "weatherCode": 4200, "windDirection": 302.31, "windGust": 2.31, "windSpeed": 1.63}}'}]

In [21]:
 second_response = client.chat.completions.create(
        model="mixtral",
        messages=messages,
    )  # get a new response from GPT where it can see the function response

In [22]:
print(second_response.choices[0].message.content)

Based on the provided data, the current weather in Silchar at the specified time is as follows:

* The temperature is 24.81°C, with an apparent temperature of 24.81°C.
* The relative humidity is 91%.
* The dew point is 20.19°C.
* The visibility is currently 4.17 km.
* The wind is coming from a direction of 302.31° with a speed of 1.63 m/s and a gust of 2.31 m/s.
* The weather code is 4200, which indicates moderate rain.
* The cloud cover is 100%, with a cloud base of 0.21 and a cloud ceiling of 0.21.
* The precipitation probability is 100%, with a rain intensity of 2.45 mm/h.
* The freezing rain intensity, sleet intensity, and snow intensity are all 0 mm/h.
* The atmospheric pressure at the surface level is 1005.51 hPa.
* The UV health concern is 1, and the UV index is 2.

I hope this information helps! Let me know if you have any other questions.


# Langchain Agents

## ReAct

In [ ]:
! pip install langchain

In [23]:
import os

from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent, tool
from langchain_openai import OpenAI

In [24]:
os.environ["OPENAI_API_BASE"] = "http://localhost:8080/v1"
os.environ["OPENAI_API_KEY"] = "NONE"

In [ ]:
! pip install langchainhub

In [29]:
prompt = hub.pull("hwchase17/react")

In [30]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [31]:
@tool
def get_word_length(word: str) -> int:
    """Returns the length of the word."""
    return len(word)

In [32]:
get_word_length.description

'get_word_length(word: str) -> int - Returns the length of the word.'

In [33]:
tools = [get_word_length]

In [34]:
llm = ChatOpenAI(model="mixtral", streaming=False)


In [35]:
agent = create_react_agent(llm, tools, prompt)


In [36]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [38]:
output = await agent_executor.ainvoke({"input": "how many letters in the word theschoolofai?"})




> Entering new AgentExecutor chain...
I need to find the length of the word "theschoolofai". I can use the `get_word_length` function to do this.

Action: get_word_length
Action Input: theschoolofai13Thought: I now know the final answer. The word "theschoolofai" has 13 letters in it.
Final Answer: The word "theschoolofai" has 13 letters in it.

> Finished chain.


## Q&A Agent


In [41]:
! pip install faiss-cpu pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 78.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 50.7 MB/s eta 0:00:00


In [43]:
! mkdir -p papers
! cd papers && wget https://arxiv.org/pdf/2401.10166.pdf && wget https://arxiv.org/ftp/arxiv/papers/2312/2312.00752.pdf && wget https://arxiv.org/pdf/2401.07519v1.pdf && wget https://arxiv.org/pdf/2212.11613v5.pdf

--2024-02-22 09:10:22--  https://arxiv.org/pdf/2401.10166.pdf
Resolving arxiv.org (arxiv.org)... 151.101.67.42, 151.101.131.42, 151.101.195.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4794587 (4.6M) [application/pdf]
Saving to: ‘2401.10166.pdf’

2401.10166.pdf      100%[===================>]   4.57M  1.47MB/s    in 3.1s    

2024-02-22 09:10:26 (1.47 MB/s) - ‘2401.10166.pdf’ saved [4794587/4794587]

--2024-02-22 09:10:26--  https://arxiv.org/ftp/arxiv/papers/2312/2312.00752.pdf
Resolving arxiv.org (arxiv.org)... 151.101.3.42, 151.101.195.42, 151.101.131.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1293868 (1.2M) [application/pdf]
Saving to: ‘2312.00752.pdf’

2312.00752.pdf      100%[===================>]   1.23M  6.19MB/s    in 0.2s    

2024-02-22 09:10:27 (6.19 MB/s) - ‘2312.00752.pdf’ saved [1293868/1293868]

--

In [45]:
! wget https://arxiv.org/pdf/2212.11613v5.pdf

--2024-02-22 09:12:01--  https://arxiv.org/pdf/2212.11613v5.pdf
Resolving arxiv.org (arxiv.org)... 151.101.3.42, 151.101.195.42, 151.101.131.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-02-22 09:12:01 ERROR 403: Forbidden.



In [1]:
import os

from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, OpenAI
from langchain.tools.retriever import create_retriever_tool
from langchain.chains import RetrievalQA
from langchain import hub

os.environ["OPENAI_API_BASE"] = "http://localhost:8080/v1"
os.environ["OPENAI_API_KEY"] = "NONE"

In [7]:
loader = PyPDFDirectoryLoader("papers/")

In [8]:
docs = loader.load()


In [9]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
db = FAISS.from_documents(texts, embeddings)

In [10]:
retriever = db.as_retriever()


In [11]:
llm = OpenAI(model="mixtral", temperature=0)


In [12]:
prompt = hub.pull("rlm/rag-prompt")


In [13]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [14]:
question = "What is VMamba? how is Mamba different from a Transformer?"
result = qa_chain.invoke({"query": question})

In [15]:
print(result["result"])

VMamba is a novel architecture for visual representation learning, inspired by the state space model. It's different from a Transformer as it achieves linear complexity without sacrificing global receptive fields, unlike Transformers which have quadratic complexity. VMamba introduces the Cross-Scan Module to address the direction-sensitive issue in handling visual data, unlike the attention mechanism in Transformers.


## Agent Chain with Search and Calculator

In [ ]:
! pip install duckduckgo-search numexpr

In [ ]:
! pip install langchain_experimental

In [19]:
import os

from langchain.chains import LLMMathChain
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_core.tools import Tool
from langchain_experimental.plan_and_execute import (
    PlanAndExecute,
    load_agent_executor,
    load_chat_planner,
)
from langchain_openai import ChatOpenAI, OpenAI

os.environ["OPENAI_API_BASE"] = "http://localhost:8080/v1"
os.environ["OPENAI_API_KEY"] = "NONE"

In [20]:
search = DuckDuckGoSearchAPIWrapper()
llm = OpenAI(model="mixtral", temperature=0)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math, but not date calculations, only math",
    ),
]

In [21]:
model = ChatOpenAI(model="mixtral", temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor)

In [24]:
output = agent.invoke(
    "Who is the current prime minister of the India and who was the first Prime Minister of India? Calculate the difference in the total years served by both the Primi Minister and who has served the maximum years?"
)



> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "Search",
  "action_input": "Current Prime Minister of India and their total years in office"
}
```


Observation: N ARENDRA MODI says India is now in the midst of amrit kaal, a golden age.Over the past ten years, the prime minister's braggadocious language has proven popular with voters. Such talk will only ... Modi is the 14th and current prime minister of India, serving since 26 May 2014 . Origins and history India follows a parliamentary system in which the prime minister is the presiding head of the government and chief of the executive of the government. Narendra Modi is the current and 14th Prime Minister of India who assumed office on 30th May 2019. Jawaharlal Nehru is the first and the longest-serving Prime Minister of India while... The President of India. Rajya Sabha. Lok Sabha. The Cabinet Secretariat. MyGov. GOI Web Directory. Press Information Bureau. National Portal of INDIA. Content managed by Prime M

ValueError: unknown format from LLM: Question: What is the cube root of 125?
```text
125**(1/3)
```
...numexpr.evaluate("125**(1/3)")...

## Agent with Coding Skills

In [25]:
from langchain import hub
from langchain.agents import AgentExecutor
from langchain_experimental.tools import PythonREPLTool
import os
os.environ["OPENAI_API_BASE"] = "http://localhost:8080/v1"
os.environ["OPENAI_API_KEY"] = "NONE"

In [26]:
tools = [PythonREPLTool()]

In [27]:
tools[0].description

'A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.'

In [28]:
from langchain.agents import create_react_agent


In [29]:
instructions = """You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
"""
base_prompt = hub.pull("langchain-ai/react-agent-template")
prompt = base_prompt.partial(instructions=instructions)

In [30]:
print(prompt.template)

{instructions}

TOOLS:
------

You have access to the following tools:

{tools}

To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```

Begin!

Previous conversation history:
{chat_history}

New input: {input}
{agent_scratchpad}


In [31]:
agent = create_react_agent(ChatOpenAI(model="mixtral", temperature=0), tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [32]:
await agent_executor.ainvoke({"input": "What is the sin(45)?"})



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


Thought: Do I need to use a tool? Yes
Action: Python_REPL
Action Input: import math
print(math.sin(math.pi/4))0.7071067811865475
Do I need to use a tool? No
Final Answer: The sine of 45 degrees, expressed in radians as math.pi/4, is approximately 0.7071067811865475.

> Finished chain.


{'input': 'What is the sin(45)?',
 'output': 'The sine of 45 degrees, expressed in radians as math.pi/4, is approximately 0.7071067811865475.'}

In [33]:
await agent_executor.ainvoke({"input" : """<p>The following iterative sequence is defined for the set of positive integers:</p>
<ul style="list-style-type:none;">
<li>$n \\to n/2$ ($n$ is even)</li>
<li>$n \\to 3n + 1$ ($n$ is odd)</li></ul>
<p>Using the rule above and starting with $13$, we generate the following sequence:
$$13 \\to 40 \\to 20 \\to 10 \\to 5 \\to 16 \\to 8 \\to 4 \\to 2 \\to 1.$$</p>
<p>It can be seen that this sequence (starting at $13$ and finishing at $1$) contains $10$ terms. Although it has not been proved yet (Collatz Problem), it is thought that all starting numbers finish at $1$.</p>
<p>Which starting number, under one million, produces the longest chain?</p>
<p class="note"><b>NOTE:</b> Once the chain starts the terms are allowed to go above one million.</p>
</p>
"""})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Python_REPL
Action Input: 
```python
def collatz(n):
    terms = 1
    while n != 1:
        if n % 2 == 0:
            n = n // 2
        else:
            n = 3*n + 1
        terms += 1
    return terms

max_terms = 0
max_number = 0
for i in range(1, 1000000):
    current_terms = collatz(i)
    if current_terms > max_terms:
        max_terms = current_terms
        max_number = i
print(max_number)
```837799
Do I need to use a tool? No
Final Answer: The starting number under one million that produces the longest chain is 837799.

> Finished chain.


{'input': '<p>The following iterative sequence is defined for the set of positive integers:</p>\n<ul style="list-style-type:none;">\n<li>$n \\to n/2$ ($n$ is even)</li>\n<li>$n \\to 3n + 1$ ($n$ is odd)</li></ul>\n<p>Using the rule above and starting with $13$, we generate the following sequence:\n$$13 \\to 40 \\to 20 \\to 10 \\to 5 \\to 16 \\to 8 \\to 4 \\to 2 \\to 1.$$</p>\n<p>It can be seen that this sequence (starting at $13$ and finishing at $1$) contains $10$ terms. Although it has not been proved yet (Collatz Problem), it is thought that all starting numbers finish at $1$.</p>\n<p>Which starting number, under one million, produces the longest chain?</p>\n<p class="note"><b>NOTE:</b> Once the chain starts the terms are allowed to go above one million.</p>\n</p>\n',
 'output': 'The starting number under one million that produces the longest chain is 837799.'}

## Agent with Google Search & Memory